In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer

# Load the CSV file
file_path = r'17072024_sales_data\Clean_Data.csv'
df = pd.read_csv(file_path, encoding='ISO-8859-1')

# Convert Ship_Date to datetime and extract features
df['Ship_Date'] = pd.to_datetime(df['Ship_Date'], errors='coerce')
df['Ship_Year'] = df['Ship_Date'].dt.year
df['Ship_Month'] = df['Ship_Date'].dt.month
df['Ship_Day'] = df['Ship_Date'].dt.day
df['Ship_Weekday'] = df['Ship_Date'].dt.weekday

# Drop the original Ship_Date column
df = df.drop(columns=['Ship_Date'])

# Updated list of features
features = ['Ship_Year', 'Ship_Month', 'Ship_Day', 'Ship_Weekday', 'Ship_Mode', 'Customer_ID', 'Segment', 'City', 'State', 'Region', 'Postal_Code', 'Product_ID', 'Category', 'Sub_Category']
target = 'Sales'

# Drop rows with missing values in the selected columns
df = df.dropna(subset=features + [target])

# Define preprocessing for numerical and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), ['Ship_Year', 'Ship_Month', 'Ship_Day', 'Ship_Weekday', 'Postal_Code']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['Ship_Mode', 'Customer_ID', 'Segment', 'City', 'State', 'Region', 'Product_ID', 'Category', 'Sub_Category'])
    ])

# Create a pipeline with preprocessing and linear regression
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Define features (X) and target (y)
X = df[features]
y = df[target]  # Use the continuous Sales variable

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R-squared (R²): {r2:.2f}")




Mean Squared Error (MSE): 421869.17
Mean Absolute Error (MAE): 251.70
R-squared (R²): 0.14


# Enrich Data

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load your data
file_path = 'Source/Enriched_Clean_Data.csv'
data = pd.read_csv(file_path, sep=';', thousands='.')

# Ensure that 'Sales' column is numeric
data['Sales'] = pd.to_numeric(data['Sales'], errors='coerce')

# Drop rows with NaN values in 'Sales' column if any
data = data.dropna(subset=['Sales'])

# Create a binary target variable
threshold = 1000  # Define your threshold for high/low sales
data['HighSales'] = (data['Sales'] > threshold).astype(int)

# Define features and target
features = [
    'Ship_Date', 'Ship_Mode', 'Customer_ID', 'Segment', 'City', 'State', 'Region', 
    'Postal_Code', 'Product_ID', 'Category', 'Sub_Category', 'shipping_costs', 
    'number_of_products_in_warehouse', 'production_costs_per_unit'
]
X = data[features]
y = data['HighSales']

# Handle categorical features
categorical_features = [
    'Ship_Mode', 'Segment', 'City', 'State', 'Region', 'Category', 'Sub_Category'
]
numerical_features = [
    'shipping_costs', 'number_of_products_in_warehouse', 'production_costs_per_unit'
]

# Preprocessing pipelines for both numeric and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Create the pipeline with preprocessing and model training
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)
y_pred_proba = pipeline.predict_proba(X_test)[:, 1]  # Probability estimates for the positive class

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Compute MSE and MAE
mse = mean_squared_error(y_test, y_pred_proba)  # Note: using probability estimates for MSE
mae = mean_absolute_error(y_test, y_pred_proba)  # Note: using probability estimates for MAE

# R² is not a typical metric for classification, but we'll calculate it anyway
r2 = r2_score(y_test, y_pred_proba)

print(f'Accuracy: {accuracy:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)
print(f'Mean Squared Error (MSE): {mse:.2f}')
print(f'Mean Absolute Error (MAE): {mae:.2f}')
print(f'R² Score: {r2:.2f}')